In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Reshape, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
NUM_CLASSES = 26
KFOLD_NUM = 7
dataset_path = '../datasets/keypoint_for_train_val.csv'

# Load dataset
data = np.loadtxt(dataset_path, delimiter=',', dtype='float32')
X_dataset = data[:, 1:]
y_dataset = data[:, 0].astype('int32')

# Define K-fold cross validation test harness
kfold = KFold(n_splits=KFOLD_NUM, shuffle=True, random_state=42)

# Function to calculate precision, recall, and F1 score
def calculate_metrics(y_true, y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    return precision, recall, f1, accuracy

# Model building function
def create_model():
    model = Sequential([
        Input(shape=(42,)),
        Reshape((21, 2, 1)),
        Conv2D(32, (3, 1), activation='relu', padding='same'),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu', padding='same'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),  # Reduced dropout
        Dense(NUM_CLASSES, activation='softmax')
    ])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1)

# Store metrics for each fold
fold_metrics = {}

# K-fold Cross Validation model evaluation
fold_no = 1
best_model_acc = 0
best_model_path = ""

for train_index, test_index in kfold.split(X_dataset):
    X_train, X_test = X_dataset[train_index], X_dataset[test_index]
    y_train, y_test = y_dataset[train_index], y_dataset[test_index]

    # Create model
    model = create_model()

    # Fit data to model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=1000,
        batch_size=32,
        callbacks=[reduce_lr, EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10), 
                   ModelCheckpoint(f'../Model_0/best_model_fold_{fold_no}.hdf5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)],
        verbose=1,
    )

    # Predictions for the fold
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Metrics calculation
    precision, recall, f1, accuracy = calculate_metrics(y_test, y_pred_classes)
    fold_metrics[fold_no] = {
        'train_precision': precision,
        'train_recall': recall,
        'train_f1': f1,
        'train_accuracy': accuracy
    }

    # Update the best model path and accuracy if the current fold's model is better
    if accuracy > best_model_acc:
        best_model_acc = accuracy
        best_model_path = f'best_model_fold_{fold_no}.hdf5'

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title(f'Fold {fold_no} - Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
    # Plot accuracy and loss for the current fold
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {fold_no} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {fold_no} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()
    
    fold_no += 1

# Print out the metrics for each fold
for fold_no, metrics in fold_metrics.items():
    print(f'Fold {fold_no}:')
    print(f"Train Precision: {metrics['train_precision']}")
    print(f"Train Recall: {metrics['train_recall']}")
    print(f"Train F1: {metrics['train_f1']}")
    print(f"Train Accuracy: {metrics['train_accuracy']}")

print(f"The best model was saved as: {best_model_path} with accuracy: {best_model_acc}")


: 

In [ ]:
import os

# Ensure the Model_0 directory exists
model_dir = '../Model_0'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Continue with your existing code...


: 